In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Iniciamos sesión local para evitar errores de memoria
spark = SparkSession.builder.appName("Fase3_Modelamiento").master("local[*]").getOrCreate()

# Cargamos TU archivo de la Fase 2
df = spark.read.parquet("/opt/spark-data/processed/secop_final_ready.parquet")

# Renombramos para que sea fácil de manejar
df_final = df.withColumnRenamed("valor_del_contrato", "label") \
             .withColumnRenamed("features_scaled", "features")

print(f"Datos cargados: {df_final.count()} registros listos.")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/14 15:53:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/02/14 15:53:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
26/02/14 15:53:31 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


Datos cargados: 967 registros listos.


26/02/14 15:53:41 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [2]:
# Dividir 70% entrenamiento y 30% prueba
train, test = df_final.randomSplit([0.7, 0.3], seed=42)

# Configurar el algoritmo
lr = LinearRegression(featuresCol="features", labelCol="label")

# Entrenar (Aquí es donde la IA aprende)
lr_model = lr.fit(train)
print("¡Modelo entrenado con éxito!")

26/02/14 15:55:05 WARN Instrumentation: [94a4de36] regParam is zero, which might cause numerical instability and overfitting.
26/02/14 15:55:06 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
26/02/14 15:55:06 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
26/02/14 15:55:06 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK
26/02/14 15:55:06 WARN Instrumentation: [94a4de36] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.


¡Modelo entrenado con éxito!


In [3]:
# Hacer las predicciones en el grupo de prueba
predictions = lr_model.transform(test)

# Calcular el error (RMSE)
evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)

print(f"📊 Error del modelo (RMSE): {rmse}")
predictions.select("label", "prediction").show(10)

📊 Error del modelo (RMSE): 2.6744819836137257
+------------+--------------------+
|       label|          prediction|
+------------+--------------------+
|   9500000.0|   9500001.185920281|
|7.31428571E8| 7.314285810888481E8|
|     2.225E7| 2.224999745346899E7|
|1.00993583E8| 1.009935830069448E8|
|       2.2E7|2.2000001219538413E7|
|  1.009953E7|1.0099527321151003E7|
|     1.552E7|1.5519998615374466E7|
|       2.8E7|  2.80000025615169E7|
|      1.84E7|1.8400001092058923E7|
|    1.0608E7|1.0608000802637849E7|
+------------+--------------------+
only showing top 10 rows

